<a href="https://colab.research.google.com/github/oraziotorre/MomentumShiftAI/blob/main/TennisDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
# Pandas per la gestione dei dati
import pandas as pd
import requests

from collections import Counter
from collections import defaultdict

# Per visualizzare i DataFrame su Google Colab in formato tabellare
from google.colab import data_table
data_table.enable_dataframe_formatter()

## Data Ingestion

Importo i csv grezzi dal MatchChartingProject su GitHub

In [64]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-2020s.csv"
response = requests.get(url)

with open("charting-m-points-2020s.csv", "w") as f:
    f.write(response.text)

dataset1 = pd.read_csv("charting-m-points-2020s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-2010s.csv"
response = requests.get(url)

with open("charting-m-points-2010s.csv", "w") as f:
    f.write(response.text)

dataset2 = pd.read_csv("charting-m-points-2010s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-to-2009.csv"
response = requests.get(url)

with open("charting-m-points-to-2009.csv", "w") as f:
    f.write(response.text)

dataset3 = pd.read_csv("charting-m-points-to-2009.csv")


dataset = pd.concat([dataset1, dataset2, dataset3], ignore_index=True)

dataset

<ipython-input-64-ee841c57e572>:7: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset1 = pd.read_csv("charting-m-points-2020s.csv")


,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0.0,0.0,0-0,1.0,True,1,6s17y3w@,NaN,192,2
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0.0,0.0,0-15,1.0,True,1,6w,6f29f2f2f3b3f2f3r2d#,159,2
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0.0,0.0,0-30,1.0,True,1,4#,NaN,189,1
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0.0,0.0,15-30,1.0,True,1,4n,4b39f3b2f1f2f1f1b3b1b1f1b1f1s2s-1b1v3f2v2m2d@,158,2
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0.0,0.0,15-40,1.0,True,1,6n,5f28f3b1s3f1d@,156,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4.0,3.0,0-40,46.0,False,2,R,NaN,NaN,1
1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5.0,3.0,0-0,47.0,False,1,S,NaN,NaN,1
1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5.0,3.0,15-0,47.0,False,1,S,NaN,NaN,1
1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5.0,3.0,30-0,47.0,False,1,S,NaN,NaN,1


### Controllo valori nulli e rari

In [65]:
print(dataset.isna().sum())

match_id         0
Pt               0
Set1             0
Set2             0
Gm1              1
Gm2              2
Pts              0
Gm#              1
TbSet            1
Svr              0
1st              0
2nd         637405
Notes       930137
PtWinner         0
dtype: int64


In [75]:
dataset['Gm1'] = dataset['Gm1'].fillna(0).astype(int)
dataset['Gm2'] = dataset['Gm2'].fillna(0).astype(int)
dataset['Gm1'] = dataset['Gm1'].fillna(0)
dataset['TbSet'] = dataset['TbSet'].fillna
dataset = dataset.drop(columns=['Gm#', 'Notes', 'Pts', 'TbSet'], errors='ignore')
print(dataset.isna().sum())

match_id         0
Pt               0
Set1             0
Set2             0
Gm1              0
Gm2              0
Svr              0
1st              0
2nd         636441
PtWinner         0
dtype: int64


In [76]:
def check_dataset(df, column_ranges=None):
    print("🔍 Controllo del dataset...\n")

    # 1. Valori nulli
    nulls = df.isnull().sum()
    nulls = nulls[nulls > 0]
    if not nulls.empty:
        print("⚠️  Colonne con valori nulli:")
        print(nulls)
    else:
        print("✅ Nessun valore nullo trovato.\n")

    # 2. Duplicati
    duplicate_rows = df.duplicated().sum()
    if duplicate_rows > 0:
        print(f"⚠️  Righe duplicate: {duplicate_rows}")
    else:
        print("✅ Nessuna riga duplicata.\n")

    # 3. Tipi di dato
    print("📊 Tipi di dato per colonna:")
    print(df.dtypes)
    print()

    # 4. Valori non numerici in colonne numeriche
    for col in df.select_dtypes(include=['object']).columns:
        try:
            pd.to_numeric(df[col])
            print(f"⚠️  La colonna '{col}' contiene stringhe ma potrebbe essere numerica.")
        except:
            pass

    # 5. Cardinalità sospetta
    print("\n🔢 Cardinalità delle colonne:")
    for col in df.columns:
        n_unique = df[col].nunique()
        if n_unique > 100 and df[col].dtype == 'object':
            print(f"⚠️  '{col}' ha {n_unique} valori unici (potrebbe contenere ID o errori)")
        else:
            print(f"{col}: {n_unique} valori unici")

    # 6. Range controlli se specificato
    if column_ranges:
        print("\n📏 Controllo range valori:")
        for col, (low, high) in column_ranges.items():
            if col in df.columns:
                out_of_bounds = df[(df[col] < low) | (df[col] > high)]
                if not out_of_bounds.empty:
                    print(f"⚠️  Colonna '{col}' ha {len(out_of_bounds)} valori fuori dal range [{low}, {high}]")
                else:
                    print(f"✅ Colonna '{col}' OK nel range [{low}, {high}]")

    print("\n✅ Controllo completato.")


check_dataset(dataset)

🔍 Controllo del dataset...

⚠️  Colonne con valori nulli:
2nd    636441
dtype: int64
⚠️  Righe duplicate: 1669
📊 Tipi di dato per colonna:
match_id    object
Pt           int64
Set1         int64
Set2         int64
Gm1          int64
Gm2          int64
Svr          int64
1st         object
2nd         object
PtWinner     int64
dtype: object


🔢 Cardinalità delle colonne:
⚠️  'match_id' ha 6002 valori unici (potrebbe contenere ID o errori)
Pt: 569 valori unici
Set1: 3 valori unici
Set2: 3 valori unici
Gm1: 26 valori unici
Gm2: 25 valori unici
Svr: 2 valori unici
⚠️  '1st' ha 273908 valori unici (potrebbe contenere ID o errori)
⚠️  '2nd' ha 214594 valori unici (potrebbe contenere ID o errori)
PtWinner: 2 valori unici

✅ Controllo completato.


Noto che ci sono valori non coerenti nelle colonne dei Set e dei Game. Decido di rimuovere tali partite

In [77]:
# Trova i match_id da escludere
invalid_match_ids = dataset.loc[dataset['Gm1'] > 60, 'match_id'].unique()

# Filtra via tutte le righe con quei match_id
dataset = dataset[~dataset['match_id'].isin(invalid_match_ids)]

In [78]:
# Trova i match_id da escludere (Set1 o Set2 > 2)
invalid_match_ids = dataset.loc[(dataset['Set1'] > 2) | (dataset['Set2'] > 2), 'match_id'].unique()

# Filtra via tutte le righe con quei match_id
dataset = dataset[~dataset['match_id'].isin(invalid_match_ids)]

Partite Rimosse: 3

### Numero di partite del dataset

In [79]:
simboli_finali = dataset['match_id'].nunique()
simboli_finali

6002

### Tipologie di esiti finali
Questi valori sono stati compararti con rispettivo significato presente sul MatchChartingProject

In [80]:
simboli_finali = Counter()


for index,row in dataset.iterrows():
  if(pd.isna(row['2nd'])):
    ultimo_carattere = row['1st'][-1]
    if(ultimo_carattere == ";"):
      ultimo_carattere = row['1st'][-2]
    simboli_finali[ultimo_carattere] += 1

  else:
    ultimo_carattere = row['2nd'][-1]
    if(ultimo_carattere == ";"):
      ultimo_carattere = row['2nd'][-2]
    simboli_finali[ultimo_carattere] += 1




for simbolo, conteggio in simboli_finali.most_common():
    print(f"'{simbolo}': {conteggio}")

'#': 339098
'*': 323028
'@': 312687
'n': 15378
'd': 15047
'S': 7541
'w': 4048
'R': 2240
'x': 889
'C': 304
'g': 97
'Q': 60
'!': 49
' ': 39
'P': 36
'e': 32
'3': 26
'1': 22
'9': 11
'8': 8
'2': 7
'+': 7
'7': 4
'=': 3
'^': 2
'b': 2
'c': 2
'4': 1
'&': 1
')': 1
'v': 1
'p': 1
'.': 1
'-': 1
'm': 1
'z': 1


### Correzione dei casi rari e enfasi a informazioni importanti in un formato più gestibile

La descrizione in formato semplice, una riga per ogni stringa:


- `1` = `noinfo_1`: il giocatore 1 vince il punto, ma non è chiaro se è ace, errore, o altro.

- `2` = `noinfo_2`: il giocatore 2 vince il punto, ma non è chiaro se è ace, errore, o altro.

- `3` = `serve_ace_1`: punto diretto con il servizio (ace) fatto dal giocatore 1.

- `4` = `serve_ace_2`: punto diretto con il servizio (ace) fatto dal giocatore 2.

- `5` = `serve_miss2_1`: il giocatore 1 vince il punto su un errore di servizio del giocatore 2.

- `6` = `serve_miss1_2`: il giocatore 2 vince il punto su un errore di servizio del giocatore 1.

- `7` = `rally_winner_1`: il giocatore 1 vince il punto con un colpo vincente durante lo scambio.

- `8` = `rally_winner_2`: il giocatore 2 vince il punto con un colpo vincente durante lo scambio.

- `9` = `rally_forced2_1`: il giocatore 1 forza un errore del giocatore 2 durante lo scambio.

- `A` = `rally_forced1_2`: il giocatore 2 forza un errore del giocatore 1 durante lo scambio.

- `B` = `rally_unforced1_2`: il giocatore 2 vince per un errore non forzato del giocatore 1.

- `C` = `rally_unforced2_1`: il giocatore 1 vince per un errore non forzato del giocatore 2.

Attualmente non ho aggiunto considerazioni sul primo e secondo servizio, e su posizionamenti della palla e tipologia di colpi giocati.

Ho considerato le challenges Hawk-Eye perse come errori forzati.
Errori forzati e winners sono stati considerati come la stessa cosa nei servizi, si potrebbe fare la stessa considerazione anche nei rally

In [81]:
for index, row in dataset.iterrows():
    winner = row["PtWinner"]

    #On first serve points
    if pd.isna(row['2nd']):
        numeri = [c for c in row['1st'] if c.isdigit()]
        ultimo = row['1st'][-1]
        if(ultimo_carattere == ";"):
          ultimo_carattere = row['1st'][-2]

        #On serve points
        if len(numeri) == 1:
            if (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 1:
                dataset.at[index, 'PointType'] = '3'
            elif (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 2:
                dataset.at[index, 'PointType'] = '4'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = '5'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 2:
                dataset.at[index, 'PointType'] = '6'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                else:
                  dataset.at[index, 'PointType'] = '2'

        #On rally points
        else:
            if ultimo == '*' and winner == 1:
                dataset.at[index, 'PointType'] = '7'
            elif ultimo == '*' and winner == 2:
                dataset.at[index, 'PointType'] = '8'
            elif (ultimo == '#' or ultimo =="C") and winner == 1:
                dataset.at[index, 'PointType'] = '9'
            elif (ultimo == '#' or ultimo =="C") and winner == 2:
                dataset.at[index, 'PointType'] = 'A'
            elif ultimo == '@' and winner == 1:
                dataset.at[index, 'PointType'] = 'B'
            elif ultimo == '@' and winner == 2:
                dataset.at[index, 'PointType'] = 'C'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                elif winner == 2:
                  dataset.at[index, 'PointType'] = '2'

    #On second serve points
    else:
        numeri = [c for c in row['2nd'] if c.isdigit()]
        ultimo = row['2nd'][-1]
        if(ultimo_carattere == ";"):
          ultimo_carattere = row['2nd'][-2]

        #On serve points
        if len(numeri) == 1:
            if (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 1:
                dataset.at[index, 'PointType'] = '3'
            elif (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 2:
                dataset.at[index, 'PointType'] = '4'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = '5'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 2:
                dataset.at[index, 'PointType'] = '6'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                else:
                  dataset.at[index, 'PointType'] = '2'

        #On rally points
        else:
            if ultimo == '*' and winner == 1:
                dataset.at[index, 'PointType'] = '7'
            elif ultimo == '*' and winner == 2:
                dataset.at[index, 'PointType'] = '8'
            elif (ultimo == '#' or ultimo =="C") and winner == 1:
                dataset.at[index, 'PointType'] = '9'
            elif (ultimo == '#' or ultimo =="C") and winner == 2:
                dataset.at[index, 'PointType'] = 'A'
            elif ultimo == '@' and winner == 1:
                dataset.at[index, 'PointType'] = 'B'
            elif ultimo == '@' and winner == 2:
                dataset.at[index, 'PointType'] = 'C'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                elif winner == 2:
                  dataset.at[index, 'PointType'] = '2'


dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Svr,1st,2nd,PtWinner,PointType
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0,0,1,6s17y3w@,NaN,2,C
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0,0,1,6w,6f29f2f2f3b3f2f3r2d#,2,A
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0,0,1,4#,NaN,1,3
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0,0,1,4n,4b39f3b2f1f2f1f1b3b1b1f1b1f1s2s-1b1v3f2v2m2d@,2,C
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0,0,1,6n,5f28f3b1s3f1d@,1,B
...,...,...,...,...,...,...,...,...,...,...,...
1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,2,R,NaN,1,1
1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,1,S,NaN,1,1
1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,1,S,NaN,1,1
1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,1,S,NaN,1,1


In [86]:
dataset = dataset.drop(columns=['1st', '2nd'], errors='ignore')

### Controllo quantità tipologie di esiti finali

In [87]:
simboli_finali = dataset['PointType'].value_counts()

simboli_finali

,count
PointType,
C,156176
B,153632
9,150425
A,146402
7,120502
8,118368
3,65780
4,60953
6,18958


### Etichettatura

Creazione Etichette sulla vittoria del set corrente ed Etichetta sulla vittoria del match corrente.
Aggiunta di dati correlati al successivo sviluppo di modelli sulla predizione dei set

In [99]:
# 1. Crea identificatore univoco del set (combinazione match_id e somma set)
dataset['SetID'] = dataset['match_id'].astype(str) + '_' + (dataset['Set1'] + dataset['Set2']).astype(str)

# 2. Conta i punti all'interno di ciascun set
dataset['PtSet'] = dataset.groupby('SetID').cumcount() + 1

# 3. Calcola il vincitore di ciascun set (ultimo punto del set)
set_winners = dataset.groupby('SetID')['PtWinner'].last()
dataset['SetWinner'] = dataset['SetID'].map(set_winners)

# 4. Calcola il vincitore del match (ultimo punto del match)
match_winners = dataset.groupby('match_id')['PtWinner'].last()
dataset['MatchWinner'] = dataset['match_id'].map(match_winners)

dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Svr,PtWinner,PointType,SetWinner,MatchWinner,PtSet,SetID,GameID,IsTiebreak,IsFifthSet
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0,0,1,2,None,2,2,1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,False,False
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0,0,1,2,None,2,2,2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,False,False
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0,0,1,1,None,2,2,3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,False,False
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0,0,1,2,None,2,2,4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,False,False
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0,0,1,1,None,2,2,5,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,2,1,None,1,1,32,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,False,True
1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,1,1,None,1,1,33,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,False,True
1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,1,1,None,1,1,34,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,False,True
1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,1,1,None,1,1,35,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,False,True


#### Analisi Bilanciamento Etichette

In [100]:
simboli_finali = dataset['SetWinner'].value_counts()

simboli_finali

,count
SetWinner,
1,522964
2,497712


In [101]:
simboli_finali = dataset['MatchWinner'].value_counts()

simboli_finali

,count
MatchWinner,
1,527368
2,493308


### Feature Engineering

Codice per distinguere la tipologia di punteggio 15-15... eccetera tenendo conto dei tiebreak e dei super tiebreak

<ipython-input-104-7703681a0eb3>:74: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dataset.groupby('GameID', group_keys=False).apply(process_game).explode().values


Creazione Feature su game point break point set point eccetera vinti e persi

Creazione delle feature derivate dai punti powered

### Data Ingestion del ranking dei giocatori

In [ ]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_players.csv"
response = requests.get(url)

with open("atp_players.csv", "w") as f:
    f.write(response.text)

playerset = pd.read_csv("atp_players.csv")

# Sostituisci NaN con stringhe vuote prima della concatenazione
first = playerset['name_first'].fillna('').str.replace(' ', '_', regex=False).str.replace('-', '_', regex=False)
last = playerset['name_last'].fillna('').str.replace(' ', '_', regex=False).str.replace('-', '_', regex=False)

# Costruisci il full_name in modo sicuro
playerset['full_name'] = (
    first + '_' + last
).str.strip('_')  # Rimuove eventuali underscore iniziali/finali se una parte è vuota
playerset['full_name'] = playerset['full_name'].str.lower()
playerset

<ipython-input-11-48ffade97733>:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  playerset = pd.read_csv("atp_players.csv")


,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id,full_name
0,100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544,gardnar_mulloy
1,100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581,pancho_segura
2,100003,Frank,Sedgman,R,19271002.0,AUS,180.0,Q962049,frank_sedgman
3,100004,Giuseppe,Merlo,R,19271011.0,ITA,NaN,Q1258752,giuseppe_merlo
4,100005,Richard,Gonzalez,R,19280509.0,USA,188.0,Q53554,richard_gonzalez
...,...,...,...,...,...,...,...,...,...
65984,213700,Matvei,Kobiakov,U,NaN,RUS,NaN,NaN,matvei_kobiakov
65985,213701,Tobia Costanzo,Baragiola Mordini,U,NaN,ITA,NaN,NaN,tobia_costanzo_baragiola_mordini
65986,213702,Dominik,Wijntjes,U,NaN,NZL,NaN,NaN,dominik_wijntjes
65987,213703,Sam,Wensley,U,NaN,AUS,NaN,NaN,sam_wensley


Prendo da GitHub i rankings degli ultimi 20 anni dei giocatori di tennis

In [ ]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_current.csv"
response = requests.get(url)

with open("atp_rankings_current.csv", "w") as f:
    f.write(response.text)

ranking1 = pd.read_csv("atp_rankings_current.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_20s.csv"
response = requests.get(url)

with open("atp_rankings_20s.csv", "w") as f:
    f.write(response.text)

ranking2 = pd.read_csv("atp_rankings_20s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_10s.csv"
response = requests.get(url)

with open("atp_rankings_10s.csv", "w") as f:
    f.write(response.text)

ranking3 = pd.read_csv("atp_rankings_10s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_00s.csv"
response = requests.get(url)

with open("atp_rankings_00s.csv", "w") as f:
    f.write(response.text)

ranking4 = pd.read_csv("atp_rankings_00s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_90s.csv"
response = requests.get(url)

with open("atp_rankings_90s.csv", "w") as f:
    f.write(response.text)

ranking5 = pd.read_csv("atp_rankings_90s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_80s.csv"
response = requests.get(url)

with open("atp_rankings_80s.csv", "w") as f:
    f.write(response.text)

ranking6 = pd.read_csv("atp_rankings_80s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_70s.csv"
response = requests.get(url)

with open("atp_rankings_70s.csv", "w") as f:
    f.write(response.text)

ranking7 = pd.read_csv("atp_rankings_70s.csv")

rankingset = pd.concat([ranking1, ranking2, ranking3, ranking4, ranking5, ranking6, ranking7], ignore_index=True)

rankingset

,ranking_date,rank,player,points
0,20240101,1,104925,11245.0
1,20240101,2,207989,8855.0
2,20240101,3,106421,7600.0
3,20240101,4,206173,6490.0
4,20240101,5,126094,4805.0
...,...,...,...,...
3292944,19771212,95,100586,NaN
3292945,19771212,97,100363,NaN
3292946,19771212,98,100176,NaN
3292947,19771212,99,100233,NaN


Ottimizzazione e Correzione degli errori presenti nel dataset originale

In [ ]:
# Crea il dizionario: full_name -> lista di player_id per ottimizzare l'esecuzione
player_id_map = defaultdict(list)

for _, row in playerset.iterrows():
  player_id_map[row['full_name']].append(row['player_id'])
  if(row['full_name']=='franko_skugor'):
    player_id_map['franco_skugor'].append(row['player_id'])
  if(row['full_name']=='jo_wilfried_tsonga'):
    player_id_map['jo_wilfred_tsonga'].append(row['player_id'])
  if(row['full_name']=='sam_groth'):
    player_id_map['samuel_groth'].append(row['player_id'])
  if(row['full_name']=='andres_artunedo_martinavarro'):
    player_id_map['andres_artunedo'].append(row['player_id'])
  if(row['full_name']=='marc_kevin_goellner'):
    player_id_map['mark_kevin_goellner'].append(row['player_id'])

rankingset['ranking_date'] = pd.to_datetime(rankingset['ranking_date'], format='%Y%m%d')

Creazione del dataset finale con i ranking dei giocatori

In [ ]:
last_match = None
match_date = None
player_id_1 = None
player_id_2 = None
ranking_1 = None
ranking_2 = None

def extract_rankings(row):
    global last_match, match_date, player_id_1, player_id_2, ranking_1, ranking_2
    match_id = row['match_id']
    if match_id == last_match:
      return pd.Series([match_date.strftime("%Y-%m-%d"), player_id_1, player_id_2, ranking_1, ranking_2])
    else:
      last_match = match_id

      match_info = match_id.split('-')
      ranking_date = match_info[0]
      name1 = match_info[-2].lower()
      name2 = match_info[-1].lower()

      # Trova i player_id in modo sicuro
      player_id1 = player_id_map.get(name1)
      player_id2 = player_id_map.get(name2)

      # Converte la data di riferimento 'ranking_date' in formato datetime
      match_date = pd.to_datetime(ranking_date, format='%Y%m%d')

      for player_id in player_id1:
          try:
              filtered_ranking = rankingset[
                  (rankingset['player'] == player_id) &
                  (rankingset['ranking_date'] <= match_date)
              ]
              if not filtered_ranking.empty:
                  ranking_1 = filtered_ranking.sort_values(by='ranking_date', ascending=False).iloc[0]['rank']
                  player_id_1 = player_id
                  break  # Esce dal ciclo appena trova un ranking valido
          except Exception as e:
              continue  # Salta in caso di errore e prova con il prossimo ID

      for player_id in player_id2:
          try:
              filtered_ranking = rankingset[
                  (rankingset['player'] == player_id) &
                  (rankingset['ranking_date'] <= match_date)
              ]
              if not filtered_ranking.empty:
                  ranking_2 = filtered_ranking.sort_values(by='ranking_date', ascending=False).iloc[0]['rank']
                  player_id_2 = player_id
                  break
          except Exception as e:
              continue

      return pd.Series([match_date.strftime("%Y-%m-%d"), player_id_1, player_id_2, ranking_1, ranking_2])


dataset[['Date', 'Player1', 'Player2', 'Ranking1', 'Ranking2']] = dataset.apply(extract_rankings, axis=1)
dataset

,match_id,GmProgression,PoweredGmProgression,Svr,Set1,Set2,Gm1,Gm2,GmWinner,SetWinner,MatchWinner,Date,Player1,Player2,Ranking1,Ranking2
0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[2, 2, 1, 2, 1, 2]","[C, A, 3, C, B, A]",1,0,0,0,0,2,2,2,2024-12-29,106329,100644,109,2
1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[2, 2, 2, 1, 1, 2]","[8, 4, 4, 5, 5, 4]",2,0,0,0,1,2,2,2,2024-12-29,106329,100644,109,2
2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[1, 1, 1, 1]","[3, 3, B, 7]",1,0,0,0,2,1,2,2,2024-12-29,106329,100644,109,2
3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[2, 2, 1, 2, 1, 2]","[8, 4, 9, C, B, 4]",2,0,0,1,2,2,2,2,2024-12-29,106329,100644,109,2
4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,"[2, 1, 2, 1, 2, 1, 1, 1]","[C, 7, C, 3, A, B, 3, 3]",1,0,0,1,3,1,2,2,2024-12-29,106329,100644,109,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118018,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,"[2, 2, 1, 1, 2, 2]","[A, A, B, 9, A, 8]",2,0,1,2,3,2,2,2,2010-01-08,103819,103786,1,6
118019,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,"[1, 1, 1, 2, 2, 1]","[9, 9, 9, C, A, 9]",1,0,1,2,4,1,2,2,2010-01-08,103819,103786,1,6
118020,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,"[2, 1, 2, 1, 2, 2]","[A, 5, C, 7, A, A]",2,0,1,3,4,2,2,2,2010-01-08,103819,103786,1,6
118021,20100108-M-Doha-SF-Roger_Federer-Nikolay_Davyd...,"[1, 1, 2, 2, 2, 1, 1, 1]","[B, 9, C, A, 8, 9, 3, 9]",1,0,1,3,5,1,2,2,2010-01-08,103819,103786,1,6


### Rimozione colonne inutili

In [ ]:
new_dataset= dataset.drop(columns=['Date'])

# Riposizionamento delle nuove colonne
is_final_index = new_dataset.columns.get_loc('GmProgression')
player1_column = new_dataset.pop('Player1')
player2_column = new_dataset.pop('Player2')
ranking1_column = new_dataset.pop('Ranking1')
ranking2_column = new_dataset.pop('Ranking2')

new_dataset.insert(is_final_index, 'Player1', player1_column)
new_dataset.insert(is_final_index + 1, 'Player2', player2_column)
new_dataset.insert(is_final_index + 2, 'Ranking1', ranking1_column)
new_dataset.insert(is_final_index + 3, 'Ranking2', ranking2_column)


new_dataset

,Player1,Player2,Ranking1,Ranking2,GmProgression,PoweredGmProgression,Svr,Set1,Set2,Gm1,Gm2,GmWinner,SetWinner,MatchWinner
0,106329,100644,109,2,"[2, 2, 1, 2, 1, 2]","[C, A, 3, C, B, A]",1,0,0,0,0,2,2,2
1,106329,100644,109,2,"[2, 2, 2, 1, 1, 2]","[8, 4, 4, 5, 5, 4]",2,0,0,0,1,2,2,2
2,106329,100644,109,2,"[1, 1, 1, 1]","[3, 3, B, 7]",1,0,0,0,2,1,2,2
3,106329,100644,109,2,"[2, 2, 1, 2, 1, 2]","[8, 4, 9, C, B, 4]",2,0,0,1,2,2,2,2
4,106329,100644,109,2,"[2, 1, 2, 1, 2, 1, 1, 1]","[C, 7, C, 3, A, B, 3, 3]",1,0,0,1,3,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118018,103819,103786,1,6,"[2, 2, 1, 1, 2, 2]","[A, A, B, 9, A, 8]",2,0,1,2,3,2,2,2
118019,103819,103786,1,6,"[1, 1, 1, 2, 2, 1]","[9, 9, 9, C, A, 9]",1,0,1,2,4,1,2,2
118020,103819,103786,1,6,"[2, 1, 2, 1, 2, 2]","[A, 5, C, 7, A, A]",2,0,1,3,4,2,2,2
118021,103819,103786,1,6,"[1, 1, 2, 2, 2, 1, 1, 1]","[B, 9, C, A, 8, 9, 3, 9]",1,0,1,3,5,1,2,2


### Generazione istanze invertite

In [ ]:
# Lista per contenere le righe modificate (invertite)
modified_rows = []

for index, row in new_dataset.iterrows():

    # Inverti 1 e 2 nella lista GmProgression
    original_progression = row['GmProgression']
    inverted_progression = [2 if x == 1 else 1 for x in original_progression]
    inverted_powered_progression = [
        '2' if x == '1' else '1' if x == '2' else
        '4' if x == '3' else '3' if x == '4' else
        '6' if x == '5' else '5' if x == '6' else
        '8' if x == '7' else '7' if x == '8' else
        'A' if x == '9' else '9' if x == 'A' else
        'C' if x == 'B' else 'B' if x == 'C' else
        x
        for x in row['PoweredGmProgression']
    ]

    # Crea una nuova riga con i dati invertiti
    new_row = {
        'GmProgression': inverted_progression,
        'PoweredGmProgression': inverted_powered_progression,
        'Svr': 2 if row['Svr'] == 1 else 1,
        'Set1': row['Set2'],
        'Set2': row['Set1'],
        'Gm1': row['Gm2'],
        'Gm2': row['Gm1'],
        'GmWinner': 2 if row['GmWinner'] == 1 else 1,
        'SetWinner': 2 if row['SetWinner'] == 1 else 1,
        'MatchWinner': 2 if row['MatchWinner'] == 1 else 1,
        'Player1': row['Player2'],
        'Player2': row['Player1'],
        'Ranking1': row['Ranking2'],
        'Ranking2': row['Ranking1']
    }

    modified_rows.append(new_row)

# Converte le nuove righe in un DataFrame
flipped_df = pd.DataFrame(modified_rows)

### Creazione Training e Test set

### Salvataggio Dataset

In [ ]:
dataset.to_csv('tennis.csv')
dataset

,Unnamed: 0,Player1,Player2,Ranking1,Ranking2,GmProgression,PoweredGmProgression,Svr,Set1,Set2,Gm1,Gm2,GmWinner,SetWinner,MatchWinner
0,0,106329,100644,109,2,"[2, 2, 1, 2, 1, 2]","['C', 'A', '3', 'C', 'B', 'A']",1,0,0,0,0,2,2,2
1,1,106329,100644,109,2,"[2, 2, 2, 1, 1, 2]","['8', '4', '4', '5', '5', '4']",2,0,0,0,1,2,2,2
2,2,106329,100644,109,2,"[1, 1, 1, 1]","['3', '3', 'B', '7']",1,0,0,0,2,1,2,2
3,3,106329,100644,109,2,"[2, 2, 1, 2, 1, 2]","['8', '4', '9', 'C', 'B', '4']",2,0,0,1,2,2,2,2
4,4,106329,100644,109,2,"[2, 1, 2, 1, 2, 1, 1, 1]","['C', '7', 'C', '3', 'A', 'B', '3', '3']",1,0,0,1,3,1,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330095,330095,100087,100113,7,11,"[2, 2, 2, 2]","['2', '2', '2', '2']",2,2,2,2,2,2,2,2
330096,330096,100087,100113,7,11,"[1, 1, 1, 1]","['1', '1', '1', '1']",1,2,2,2,3,1,2,2
330097,330097,100087,100113,7,11,"[2, 2, 2, 2]","['2', '2', '2', '2']",2,2,2,3,3,2,2,2
330098,330098,100087,100113,7,11,"[2, 2, 2, 2]","['2', '2', '2', '2']",2,2,2,3,4,2,2,2
